In [17]:
# Colab OS에 Java (OpenJDK 11) 설치 (KonlPy 실행 엔진)
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"

# 한국어 형태소 분석기(KoNLPy) 및 Mecab 설치
!pip install konlpy
!pip install "mecab-python3==1.0.0"
!bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)

print("✅ Java 및 KoNLPy/Mecab 설치 완료.")
print("   -> [중요] '런타임 > 런타임 다시 시작'을 수행한 후 [셀 2]부터 실행하세요.")

^C
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/cli/base_command.py", line 179, in exc_logging_wrapper
    status = run_func(*args)
             ^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/cli/req_command.py", line 67, in wrapper
    return func(self, options, args)
           ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/commands/install.py", line 447, in run
    conflicts = self._determine_conflicts(to_install)
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/commands/install.py", line 578, in _determine_conflicts
    return check_install_conflicts(to_install)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/operations/check.py", line 101, in check_install_conflicts
    package_set, _ = create_package_set_from_installed()
           

In [11]:
import pandas as pd
import numpy as np
import os
import pickle
import warnings
from google.colab import drive

# ML 라이브러리
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from konlpy.tag import Okt

# K-Means n_init 관련 경고 메시지 무시
warnings.filterwarnings('ignore', category=FutureWarning)

In [ ]:
# --- 1. 경로 설정 및 드라이브 마운트 ---
print("Google Drive 마운트 중...")
drive.mount('/content/drive')

# 3번 노트북에서 모든 정보(id, title, sigungucode, overview)를
# 병합하여 저장한 최종 상세 데이터 파일을 '입력'으로 사용합니다.
BASE_PATH = "/content/drive/MyDrive/D-up_Fest_compass/중간발표 코랩/data/"
INPUT_KOR_CSV_PATH = os.path.join(BASE_PATH, "seoul_festivals_kor_total.csv") # 👈 정확한 파일명 명시

# 5_FastAPI 서버가 사용할 파일 (출력)
API_DATA_PATH = "/content/drive/MyDrive/D-up_Fest_compass/backend/app/data/"
API_MODEL_PATH = "/content/drive/MyDrive/D-up_Fest_compass/backend/app/models/"

# FastAPI 서버용 폴더 생성 (없을 경우)
os.makedirs(API_DATA_PATH, exist_ok=True)
os.makedirs(API_MODEL_PATH, exist_ok=True)

print(f"입력 파일: {INPUT_KOR_CSV_PATH}")
print(f"출력 경로 (Data): {API_DATA_PATH}")
print(f"출력 경로 (Model): {API_MODEL_PATH}")


Google Drive 마운트 중...
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
입력 파일: /content/drive/MyDrive/D-up_Fest_compass/중간발표 코랩/data/seoul_festivals_kor_total.csv
출력 경로 (Data): /content/drive/MyDrive/D-up_Fest_compass/backend/app/data/
출력 경로 (Model): /content/drive/MyDrive/D-up_Fest_compass/backend/app/models/


In [18]:
# --- 2. 한국어 토크나이저 정의 (contents_analysis.ipynb와 동일) ---
try:
    okt = Okt()
    print("Okt (한국어 형태소 분석기) 로드 성공.")
except Exception as e:
    print(f"❌ Okt 로드 실패: {e}")
    print("   -> [런타임] > [런타임 다시 시작] 후 [셀 2]부터 다시 실행하세요.")

def kor_tokenizer(text: str) -> list:
    """한국어 '명사' 중 두 글자 이상만 추출하는 Tokenizer"""
    try:
        nouns = okt.nouns(str(text))
        return [noun for noun in nouns if len(noun) > 1] # 한 글자 명사 제거
    except Exception:
        return []

Okt (한국어 형태소 분석기) 로드 성공.


In [19]:
# --- 3. 데이터 로드 및 전처리 ---
try:
    df_ml = pd.read_csv(INPUT_KOR_CSV_PATH, dtype={'contentid': str})
    print(f"\n원본 데이터 로드 성공: {len(df_ml)}건")
except FileNotFoundError:
    print(f"❌ [에러] 입력 파일 '{INPUT_KOR_CSV_PATH}'를 찾을 수 없습니다.")
    # 3_dom_for_TourAPI_collect.ipynb가 정상 실행되었는지 확인하세요.
    raise

# 3-1. ML 분석에 사용할 데이터 정제 (overview가 있는 데이터만)
df_ml = df_ml.dropna(subset=['overview'])
# (선택) overview 내용이 짧은 데이터 제거 (예: 10글자 미만)
df_ml = df_ml[df_ml['overview'].str.len() > 10].copy()
df_ml.reset_index(drop=True, inplace=True) # 👈 [중요] 인덱스 재설정

print(f"정제 완료 (유효 데이터): {len(df_ml)}건")

# 3-2. FastAPI 서버가 사용할 마스터 데이터베이스 생성
# (입력 파일에 4개 컬럼이 모두 있으므로 바로 추출)
festivals_db = df_ml[['contentid', 'title', 'overview', 'sigungucode']].copy()

# 3-3. contentid <-> matrix index 맵핑 딕셔너리 생성
# (FastAPI에서 contentid로 유사도 행렬을 빠르게 조회하기 위함)
contentid_to_index = pd.Series(df_ml.index, index=df_ml['contentid']).to_dict()


원본 데이터 로드 성공: 239건
정제 완료 (유효 데이터): 239건


In [20]:
# --- 4. TF-IDF 벡터화 및 코사인 유사도 계산 ---
print("\nTF-IDF 벡터화 및 코사인 유사도 계산 시작...")

# 4-1. TF-IDF Vectorizer 학습
tfidf_vectorizer = TfidfVectorizer(
    min_df=2,          # 최소 2개 이상 문서에 등장한 단어만 사용
    tokenizer=kor_tokenizer
)
tfidf_matrix = tfidf_vectorizer.fit_transform(df_ml['overview'])

print(f"TF-IDF Matrix 생성 완료: (행사 수: {tfidf_matrix.shape[0]}, 단어 수: {tfidf_matrix.shape[1]})")

# 4-2. 코사인 유사도 행렬 계산
cosine_sim_matrix = cosine_similarity(tfidf_matrix, tfidf_matrix)

print(f"코사인 유사도 Matrix 생성 완료: {cosine_sim_matrix.shape}")


TF-IDF 벡터화 및 코사인 유사도 계산 시작...


/usr/local/lib/python3.12/dist-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


TF-IDF Matrix 생성 완료: (행사 수: 239, 단어 수: 1205)
코사인 유사도 Matrix 생성 완료: (239, 239)


In [21]:
cosine_sim_matrix

array([[1.        , 0.36236275, 0.08969315, ..., 0.08061907, 0.0443188 ,
        0.08577615],
       [0.36236275, 1.        , 0.01186274, ..., 0.01200705, 0.04939401,
        0.03102813],
       [0.08969315, 0.01186274, 1.        , ..., 0.1856442 , 0.04819844,
        0.09641604],
       ...,
       [0.08061907, 0.01200705, 0.1856442 , ..., 1.        , 0.03634667,
        0.02263819],
       [0.0443188 , 0.04939401, 0.04819844, ..., 0.03634667, 1.        ,
        0.0537958 ],
       [0.08577615, 0.03102813, 0.09641604, ..., 0.02263819, 0.0537958 ,
        1.        ]])

In [22]:
# --- 5. FastAPI 서버용 산출물 저장 ---
# (아키텍처 다이어그램의 'Offline' 산출물)

print("\nFastAPI 서버용 산출물 저장 중...")

# 1. 마스터 데이터 (festivals_db.csv)
FESTIVALS_DB_PATH = os.path.join(API_DATA_PATH, "festivals_db.csv")
festivals_db.to_csv(FESTIVALS_DB_PATH, index=False, encoding='utf-8-sig')
print(f"✅ 1. 마스터 DB 저장 완료: {FESTIVALS_DB_PATH}")

# 2. 코사인 유사도 행렬 (cosine_sim_matrix.pkl)
COSINE_SIM_PATH = os.path.join(API_MODEL_PATH, "cosine_sim_matrix.pkl")
with open(COSINE_SIM_PATH, "wb") as f:
    pickle.dump(cosine_sim_matrix, f)
print(f"✅ 2. 코사인 유사도 행렬 저장 완료: {COSINE_SIM_PATH}")

# 3. ID-Index 맵핑 (contentid_to_index.pkl)
ID_MAP_PATH = os.path.join(API_MODEL_PATH, "contentid_to_index.pkl")
with open(ID_MAP_PATH, "wb") as f:
    pickle.dump(contentid_to_index, f)
print(f"✅ 3. ID-Index 맵핑 저장 완료: {ID_MAP_PATH}")

# 4. TF-IDF 벡터라이저 (tfidf_vectorizer.pkl)
TFIDF_VEC_PATH = os.path.join(API_MODEL_PATH, "tfidf_vectorizer.pkl")
with open(TFIDF_VEC_PATH, "wb") as f:
    pickle.dump(tfidf_vectorizer, f)
print(f"✅ 4. TF-IDF 벡터라이저 저장 완료: {TFIDF_VEC_PATH}")

print("\n--- 모든 ML 모델링 및 파일 저장이 완료되었습니다. ---")


FastAPI 서버용 산출물 저장 중...
✅ 1. 마스터 DB 저장 완료: /content/drive/MyDrive/D-up_Fest_compass/backend/app/data/festivals_db.csv
✅ 2. 코사인 유사도 행렬 저장 완료: /content/drive/MyDrive/D-up_Fest_compass/backend/app/models/cosine_sim_matrix.pkl
✅ 3. ID-Index 맵핑 저장 완료: /content/drive/MyDrive/D-up_Fest_compass/backend/app/models/contentid_to_index.pkl
✅ 4. TF-IDF 벡터라이저 저장 완료: /content/drive/MyDrive/D-up_Fest_compass/backend/app/models/tfidf_vectorizer.pkl

--- 모든 ML 모델링 및 파일 저장이 완료되었습니다. ---
